# Explore here

It's recommended to use this notebook for exploration purposes.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot

In [11]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv"
df_raw = pd.read_csv(url, sep = ";")

In [5]:
df_raw.info()
#Info sobre data types y nulls 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [8]:
df_raw.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


* Drop duplicate rows:

In [23]:
df_raw = df_raw.drop_duplicates()

* Chequeamos en cada variable de type 'object' si hay 'unknown':

In [24]:
df_raw['job'].value_counts()

admin.           10419
blue-collar       9253
technician        6739
services          3967
management        2924
retired           1718
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [25]:
df_raw['marital'].value_counts()

married     24921
single      11564
divorced     4611
unknown        80
Name: marital, dtype: int64

In [26]:
df_raw['education'].value_counts()

university.degree      12164
high.school             9512
basic.9y                6045
professional.course     5240
basic.4y                4176
basic.6y                2291
unknown                 1730
illiterate                18
Name: education, dtype: int64

In [27]:
df_raw['default'].value_counts()

no         32577
unknown     8596
yes            3
Name: default, dtype: int64

In [28]:
df_raw['housing'].value_counts()

yes        21571
no         18615
unknown      990
Name: housing, dtype: int64

In [29]:
df_raw['loan'].value_counts()

no         33938
yes         6248
unknown      990
Name: loan, dtype: int64

In [30]:
df_raw['contact'].value_counts()

cellular     26135
telephone    15041
Name: contact, dtype: int64

In [18]:
df_raw['month'].value_counts()

may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64

In [31]:
df_raw['day_of_week'].value_counts()

thu    8618
mon    8512
wed    8134
tue    8086
fri    7826
Name: day_of_week, dtype: int64

In [32]:
df_raw['poutcome'].value_counts()

nonexistent    35551
failure         4252
success         1373
Name: poutcome, dtype: int64

In [33]:
df_raw['y'].value_counts()

no     36537
yes     4639
Name: y, dtype: int64

* Reemplazamos los unknown por los valores más frecuentes en las variables categóricas:

In [43]:
df_raw.loc[df_raw["marital"] == "unknown", "marital"] = "married"
df_raw.loc[df_raw["job"] == "unknown", "job"] = "admin."
df_raw.loc[df_raw["education"] == "unknown", "education"] = "university.degree"
df_raw.loc[df_raw["default"] == "unknown", "default"] = "no"
df_raw.loc[df_raw["housing"] == "unknown", "housing"] = "yes"
df_raw.loc[df_raw["loan"] == "unknown", "loan"] = "no"

In [ ]:
#Lo del chunk anterior también se puede hacer con boolean condition
#Just leaving it here for educational purposes :)

#boolean_condition = df.marital == 'Unknown'
#column_name = "marital"
#new_value = "married"

#dfraw.loc[boolean_condition, column_name] = new_value

* Reemplazamos los unknown por la media en las variables numéricas:

In [ ]:
#Creating a function that works in all those categories is a good practice.

In [44]:
df_raw['age'].unique()

array([56, 57, 37, 40, 45, 59, 41, 24, 25, 29, 35, 54, 46, 50, 39, 30, 55,
       49, 34, 52, 58, 32, 38, 44, 42, 60, 53, 47, 51, 48, 33, 31, 43, 36,
       28, 27, 26, 22, 23, 20, 21, 61, 19, 18, 70, 66, 76, 67, 73, 88, 95,
       77, 68, 75, 63, 80, 62, 65, 72, 82, 64, 71, 69, 78, 85, 79, 83, 81,
       74, 17, 87, 91, 86, 98, 94, 84, 92, 89])

In [45]:
list(df_raw['duration'].unique()) #Abrimos como text editor, control F y buscamos unknown

[261,
 149,
 226,
 151,
 307,
 198,
 139,
 217,
 380,
 50,
 55,
 222,
 137,
 293,
 146,
 174,
 312,
 440,
 353,
 195,
 38,
 262,
 342,
 181,
 172,
 99,
 93,
 233,
 255,
 362,
 348,
 386,
 73,
 230,
 208,
 336,
 365,
 1666,
 577,
 366,
 314,
 160,
 212,
 188,
 22,
 616,
 178,
 355,
 225,
 266,
 253,
 179,
 269,
 135,
 161,
 787,
 145,
 449,
 812,
 164,
 357,
 232,
 91,
 273,
 158,
 177,
 200,
 176,
 211,
 214,
 1575,
 349,
 337,
 272,
 193,
 165,
 1042,
 20,
 246,
 529,
 192,
 1467,
 180,
 48,
 213,
 545,
 583,
 221,
 426,
 287,
 197,
 257,
 229,
 400,
 190,
 21,
 300,
 123,
 325,
 514,
 849,
 194,
 286,
 247,
 518,
 364,
 98,
 439,
 79,
 175,
 61,
 78,
 102,
 579,
 143,
 677,
 267,
 345,
 185,
 207,
 69,
 100,
 125,
 461,
 240,
 70,
 136,
 528,
 541,
 338,
 163,
 87,
 301,
 46,
 52,
 204,
 155,
 71,
 243,
 186,
 559,
 2033,
 85,
 506,
 114,
 843,
 427,
 292,
 128,
 107,
 303,
 81,
 270,
 228,
 673,
 250,
 130,
 252,
 138,
 412,
 19,
 717,
 313,
 289,
 683,
 1077,
 167,
 356,
 277,
 218

In [46]:
df_raw['duration'].unique()

array([ 261,  149,  226, ..., 1246, 1556, 1868])

* Convertir age en categorial con grupos de edad de 10 años: 

In [3]:
df_raw['age_bins'] = pd.cut(x=df_raw['age'], bins=[10,20,30,40,50,60,70,80,90,100])
df_raw[['age_bins','age']].head()

,age_bins,age
0,"(50, 60]",56
1,"(50, 60]",57
2,"(30, 40]",37
3,"(30, 40]",40
4,"(50, 60]",56


* Insertamos las categorías 'basic.9y','basic.6y','basic4y' en 'middle_school':

In [4]:
df_raw['education'] = df_raw['education'].replace({'basic.9y': 'middle_school', 'basic.6y': 'middle_school', 'basic.4y': 'middle_school'})


* Convertir la target variable en dummy:

In [12]:
df_one = pd.get_dummies(df_raw["y"])
print(df_one)

       no  yes
0       1    0
1       1    0
2       1    0
3       1    0
4       1    0
...    ..  ...
41183   0    1
41184   1    0
41185   1    0
41186   0    1
41187   1    0

[41188 rows x 2 columns]


In [13]:
df_raw = pd.concat((df_one, df_raw), axis=1)
df_raw = df_raw.drop(["no"], axis=1)
df_raw = df_raw.drop(["y"], axis=1)
result = df_raw.rename(columns={"yes": "y"})
print(result)

       y  age          job  marital            education  default housing  \
0      0   56    housemaid  married             basic.4y       no      no   
1      0   57     services  married          high.school  unknown      no   
2      0   37     services  married          high.school       no     yes   
3      0   40       admin.  married             basic.6y       no      no   
4      0   56     services  married          high.school       no      no   
...   ..  ...          ...      ...                  ...      ...     ...   
41183  1   73      retired  married  professional.course       no     yes   
41184  0   46  blue-collar  married  professional.course       no      no   
41185  0   56      retired  married    university.degree       no     yes   
41186  1   44   technician  married  professional.course       no      no   
41187  0   74      retired  married  professional.course       no     yes   

      loan    contact month  ... duration  campaign  pdays  previous  \
0  